In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
import pandas as pd
df=pd.DataFrame(pd.read_excel("/content/drive/MyDrive/drive-download-20240718T011138Z-001/Input.xlsx"))
df.head()


,URL_ID,URL
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...
3,bctech2014,https://insights.blackcoffer.com/effective-man...
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...


In [4]:
import pandas as pd
import numpy as np
import nltk
import re
from bs4 import BeautifulSoup as bs
import requests
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import SyllableTokenizer
lem=WordNetLemmatizer()
import warnings
warnings.filterwarnings("ignore")

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
stop_words=stopwords.words("english")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [5]:
class data_ingestion:

  def primary(self):
    try:
      data=pd.DataFrame(pd.read_excel("/content/drive/MyDrive/drive-download-20240718T011138Z-001/Input.xlsx"))
      return data
    except Exception as e:
      print(f"Error {e}")

  def secondary(self):
    data=pd.DataFrame(pd.read_excel("/content/drive/MyDrive/drive-download-20240718T011138Z-001/Input.xlsx"))
    df=data.copy()
    updated_list=[]
    No_Matching_Data=[]
    Blank_link={}

    for i,url in enumerate(df["URL"]):
       response_code=requests.get(url)
       soup=bs(response_code.content,"html.parser")
       article_title=soup.find("title").text
       article_text=soup.find("div",class_="td-post-content tagdiv-type")

       if article_text is not None:
        all_text=article_text.get_text(strip=True,separator="\n")
        firstdata=all_text.splitlines()

       else:
        print(f"No matching element found in the HTML for URL:{url}")
        firstdata=[]
        Blank_link[f"URL_{i+1}"]=url
        Blank={
               "URL_ID":f"URL_{i+1}",
               "URL":url
          }
        No_Matching_Data.append(Blank)

       new_dataframe={
          "URL_ID":df["URL_ID"][i],
          "URL":url,
          "article_words":f"{article_title}-{firstdata}"
         }
       updated_list.append(new_dataframe)

       filename=df["URL_ID"][i]
       space=" "
       filepath="/content/drive/MyDrive/drive-download-20240718T011138Z-001"

       file1=open(f"{filepath}/{filename}.txt","w",encoding="utf-8")
       file1.writelines(article_title)
       file1.writelines(space)
       file1.writelines(firstdata)
       file1.close()
    df=pd.DataFrame(updated_list)
    df.to_excel("/content/drive/MyDrive/drive-download-20240718T011138Z-001/Output.xlsx")
    return df


In [6]:
updated_df=pd.DataFrame(pd.read_excel("/content/drive/MyDrive/drive-download-20240718T011138Z-001/Output.xlsx",index_col=0))
updated_df.head()

,URL_ID,URL,article_words
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML and AI-based insurance premium model to pre...
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,Streamlined Integration: Interactive Brokers A...
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,Efficient Data Integration and User-Friendly I...
3,bctech2014,https://insights.blackcoffer.com/effective-man...,Effective Management of Social Media Data Extr...
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,Streamlined Trading Operations Interface for M...


In [39]:
class Analysis:

   def Stopward_data(self) :
     stopword_auditor=open("/content/drive/MyDrive/drive-download-20240718T011138Z-001/StopWords/StopWords_Auditor.txt")
     stopword_currencies=open("/content/drive/MyDrive/drive-download-20240718T011138Z-001/StopWords/StopWords_Currencies.txt")
     stopword_DatesandNumbers=open("/content/drive/MyDrive/drive-download-20240718T011138Z-001/StopWords/StopWords_DatesandNumbers.txt")
     stopword_Generic=open("/content/drive/MyDrive/drive-download-20240718T011138Z-001/StopWords/StopWords_Generic.txt")
     stopword_GenericLong=open("/content/drive/MyDrive/drive-download-20240718T011138Z-001/StopWords/StopWords_GenericLong.txt")
     stopword_Geographic=open("/content/drive/MyDrive/drive-download-20240718T011138Z-001/StopWords/StopWords_Geographic.txt")
     stopword_Names=open("/content/drive/MyDrive/drive-download-20240718T011138Z-001/StopWords/StopWords_Names.txt")
     return stopword_auditor,stopword_currencies,stopword_DatesandNumbers,stopword_Generic,stopword_GenericLong,stopword_Geographic,stopword_Names

   def text_corpus(self,x):

      string_format=str(x).lower()
      lower_words=re.sub('[^a-zA-Z]+',' ',string_format).strip()
      token=word_tokenize(lower_words)
      token_word=[t for t in token if t not in (stopword_auditor,stopword_currencies,stopword_DatesandNumbers,stopword_Generic,stopword_GenericLong,stopword_Geographic,stopword_Names)]
      lemmatizzed=[lem.lemmatize(w) for w in token_word]
      return lemmatizzed

   def MasterDictionary_data(self):
     #Postive Dictionary
     positive_file=open("/content/drive/MyDrive/drive-download-20240718T011138Z-001/MasterDictionary/positive-words.txt",encoding="utf-8")
     positive_file.seek(0)
     positive_words=positive_file.read().split()

     #Negative Dictionary
     negative_file=open("/content/drive/MyDrive/drive-download-20240718T011138Z-001/MasterDictionary/negative-words.txt",encoding="latin-1")
     negative_file.seek(0)
     negative_words=negative_file.read().split()
     return positive_words,negative_words

   def count_syllables(self,x):
     word=str(x)
     vowels="aeiouy"
     exceptions=["es","ed"]
     count=0
     previous_char_was_vowel=False

     for exception in exceptions:
       if word.endswith(exception):
        return count

     for char in word.lower():
      if char in vowels:
       if not previous_char_was_vowel:
        count+=1
        previous_char_was_vowel=True
       else:
        previous_char_was_vowel=False
     return count

   def count_complex_words(self,x):
      tokenizer = SyllableTokenizer()
      words=x
      count = 0
      for word in words:
        syllables = tokenizer.tokenize(word)
        if len(syllables) > 2:
          count += 1
      return count

   def calculate_complexity_percentage(self,words):
       num_complex_words = self.count_complex_words(words)
       total_words = len(words)
       complexity_percentage = (num_complex_words / total_words) * 100
       return complexity_percentage

   def count_syllables_per_word(self,words):
       syllables_per_word={word:self.count_syllables(word)for word in words}
       return syllables_per_word

   def personal_pronoun_count(self,word_list):
       list_of_words=["i","we","my","ours","us"]
       count=0
       for word in word_list:
         if word in list_of_words:
           count+=1
       return count

   def Average_Word_Count(self,word_list):
      count=0
      for i in word_list:
        for j in i:
          count+=1
      return count/len(word_list)



In [40]:
class Col_Structure(Analysis):

  def Col_Structure_Primary(self,data):
    #Output_data=[]
    updated_list=[]

    for i,j,column in zip(data["URL_ID"],data["URL"],data["article_words"]):
      #Returns A Tokenized words
      preprocessed_word=self.text_corpus(column)

      #Existing Dictionary in Text file
      positive_dictionary,negative_dictionary=self.MasterDictionary_data()

      #1.Postive Score
      positive_count=[]
      for ps_words in preprocessed_word:
        if ps_words in positive_dictionary:
          positive_count.append(ps_words)
      positive_score=len(positive_count)

      #2.Negative Score
      negative_count=[]
      for ng_words in preprocessed_word:
        if ng_words in negative_dictionary:
          negative_count.append(ng_words)
      negative_score=len(negative_count)

      #3.Polarity Score
      polarity_score=(positive_score-negative_score)/((positive_score+negative_score)+0.000001)

      #4.Subjectivity Score
      subjective_score=(positive_score + negative_score) / ((len(preprocessed_word))+0.000001)

      #5.Avg Sentence Length
      total_sentences=len(nltk.tokenize.sent_tokenize(column))
      average_sentence_length=round(len(preprocessed_word)/total_sentences,0)

      #6.Percentage of Complex Words
      Percentage_of_Complex_words=self.calculate_complexity_percentage(preprocessed_word)

      #7.FOG Index
      fog_index=0.4*(average_sentence_length+Percentage_of_Complex_words)

      #8.Avg Number of Words per Sentence
      Average_Number_of_Words_per_Sentence=round(len(column.split())/total_sentences,0)

      #9.Complex Word Count
      complex_word_count=self.count_complex_words(preprocessed_word)

      #10.Word Count
      word_count=len(preprocessed_word)

      #11.Syllable Per Word
      syllable_per_word=self.count_syllables_per_word(preprocessed_word)

      #12.Personal Pronouns
      personal_pronouns=self.personal_pronoun_count(preprocessed_word)

      #13.Average Word Length
      average_word_length=round(self.Average_Word_Count(preprocessed_word),0)

      final_dict={
          "URL_ID":i,
          "URL":j,
          "article_words":column,
          "POSITIVE_SCORE":positive_score,
          "NEGATIVE_SCORE":negative_score,
          "POLARITY_SCORE":polarity_score,
          "SUBJECTIVITY_SCORE":subjective_score,
          "AVG_SENTENCE_LENGTH":average_sentence_length,
          "PERCENTAGE_OF_COMPLEX_WORDS":Percentage_of_Complex_words,
          "FOG_INDEX":fog_index,
          "AVG_NUMBER_OF_WORDS_PER_SENTENCE":Average_Number_of_Words_per_Sentence,
          "COMPLEX_WORD_COUNT":complex_word_count,
          "WORD_COUNT":word_count,
          "SYLLABLE_PER_WORD":syllable_per_word,
          "PERSONAL_PRONOUNS":personal_pronouns,
          "AVG_WORD_LENGTH":average_word_length
      }
      updated_list.append(final_dict)

    Output_data=pd.DataFrame(updated_list)
    Output_data.to_excel("/content/drive/MyDrive/drive-download-20240718T011138Z-001/Output Data Structures(2).xlsx")

    return Output_data


In [41]:
col=Col_Structure()
col.Col_Structure_Primary(updated_df)

,URL_ID,URL,article_words,POSITIVE_SCORE,NEGATIVE_SCORE,POLARITY_SCORE,SUBJECTIVITY_SCORE,AVG_SENTENCE_LENGTH,PERCENTAGE_OF_COMPLEX_WORDS,FOG_INDEX,AVG_NUMBER_OF_WORDS_PER_SENTENCE,COMPLEX_WORD_COUNT,WORD_COUNT,SYLLABLE_PER_WORD,PERSONAL_PRONOUNS,AVG_WORD_LENGTH
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML and AI-based insurance premium model to pre...,144,47,0.507853,0.073631,15.0,39.861218,21.944487,15.0,1034,2594,"{'ml': 0, 'and': 1, 'ai': 1, 'based': 0, 'insu...",2,6.0
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,Streamlined Integration: Interactive Brokers A...,22,6,0.571429,0.054795,10.0,33.855186,17.542074,10.0,173,511,"{'streamlined': 0, 'integration': 3, 'interact...",1,6.0
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,Efficient Data Integration and User-Friendly I...,24,10,0.411765,0.055195,18.0,31.168831,19.667532,17.0,192,616,"{'efficient': 2, 'data': 1, 'integration': 3, ...",1,6.0
3,bctech2014,https://insights.blackcoffer.com/effective-man...,Effective Management of Social Media Data Extr...,15,7,0.363636,0.045361,9.0,35.051546,17.620619,9.0,170,485,"{'effective': 2, 'management': 2, 'of': 1, 'so...",1,6.0
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,Streamlined Trading Operations Interface for M...,18,3,0.714286,0.035117,21.0,29.933110,20.373244,20.0,179,598,"{'streamlined': 0, 'trading': 1, 'operation': ...",1,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,bctech2153,https://insights.blackcoffer.com/population-an...,Population and Community Survey of America - B...,15,10,0.200000,0.028345,25.0,26.417234,20.566893,27.0,233,882,"{'population': 3, 'and': 1, 'community': 2, 's...",3,5.0
143,bctech2154,https://insights.blackcoffer.com/google-lsa-ap...,Google LSA API Data Automation and Dashboardin...,24,20,0.090909,0.031496,21.0,23.192555,17.677022,21.0,324,1397,"{'google': 2, 'lsa': 1, 'api': 1, 'data': 1, '...",7,5.0
144,bctech2155,https://insights.blackcoffer.com/healthcare-da...,Healthcare Data Analysis - Blackcoffer Insight...,18,13,0.161290,0.072941,25.0,14.352941,15.741176,24.0,61,425,"{'healthcare': 2, 'data': 1, 'analysis': 2, 'b...",12,5.0
145,bctech2156,https://insights.blackcoffer.com/budget-sales-...,"Budget, Sales KPI Dashboard using Power BI - B...",1,0,0.999999,0.008065,124.0,20.967742,57.987097,149.0,26,124,"{'budget': 1, 'sale': 1, 'kpi': 1, 'dashboard'...",0,6.0
